#  Modeling and feature importance visual

## Feature Selection

In this section, I will select specific features for model training. The original dataset contains 265 features, most of which are categorical attributes, dominantly the list of drug names that are approved as pain medications that need special AED licensed prescriber.Here,my main target is to predict the overdoses using the  the list of prescriber specialists with NPI information and the FDA approved drug lists. The data set has only total opiod cliams and total claim counts and i took the standard prescription for opiod drugs is 84 days which is 12 weeks. Therefore, an additional feature was created in the data cleaning, pre processing that shows avg_op_supply which i limit to the standard to be 84 and any ones who prescribes the category of medications under the controlled substance act(CSA). The CSA is used for organizing drugs based on the risk of abuse or harm to users in five and my capestone focuses on the approved by [FDA](https://www.fda.gov/drugs/drugsafety/informationbydrugclass/ucm251735.htm) and [DEA](https://www.deadiversion.usdoj.gov/schedules/) to be prescribed by licensed and certified health professional.  These drug lists that are considered controlled substances under the Controlled Substances Act (CSA) are divided into five schedules.  An updated and complete list of the schedules is published annually in [Title 21 Code of Federal Regulations (C.F.R.) §§ 1308.11 through 1308.15](https://www.deadiversion.usdoj.gov/21cfr/cfr/2108cfrt.htm).

Therfore, for analysis and visualization  i only took those physicians who have greater opood cliams from the center for medical services dataset and created a feature average opiod supply  by dividing the total cliam by the opiod cliam and reviewed those specialists who prescribe more than the set limit of 12 weeks and that also become a new feature that measures the length of opiod prescription supply more than 84 days. As been visualized in notebook 1.3, significant amount of prescribers are family physicians, followed by internal medicine,nurse practitioners, Physicain Assistant and physical medicine and rehabilitation. When i group based on speciality and the vsuals can be accessed   [here top 20](https://plot.ly/~kiros/112) and the lists are as follows:-
  > $'Cardiology', 'Nephrology', 'Endocrinology', 'Neurology'  'General Practice', 'Internal Medicine', 'Urology'$,   > $'Pulmonary Disease', 'Gastroenterology', 'Family Practice', 'Infectious Disease','Psychiatry'$ 
  >$'Nurse Practitioner', 'Ophthalmology','Allergy/Immunology', 'Otolaryngology', 'Psychiatry & Neurology'$ , $'Rheumatology', 'Dermatology', 'Obstetrics/Gynecology'$

In [23]:
# importing necessary libraries and packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
np.random.seed(2018)

# Configure visual settings:
plt.style.use('fivethirtyeight')
sns.set(font_scale=2)
cmap = sns.diverging_palette(220, 10, as_cmap=True) # one of the many color mappings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10.0, 8.0) 
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [24]:
## Read Data
df=pd.read_csv('./inputs/cleaned_prescriber_info18.csv')
df.head()[:2]

/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,NPI,Gender,State,Credentials,Specialty,NPPES Provider Last Name,NPPES Provider First Name,NPPES Provider ZIP Code,NPPES Provider State,Totalclaimcount,...,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,avg_op_supply,op_longer,Opioid_Prescriber,op_prescriber
0,1003002320,M,MS,D.M.D. P.A.,Dentist,EKLUND,D.,39208.0,MS,129,...,0,0,0,0,0,0,4.5,1.0,1,1.0
1,1003004771,F,CO,M.D.,Pulmonary Disease,RIGGLE,ERIN,80120.0,CO,537,...,0,0,16,0,0,0,4.5,0.0,0,0.0


In [25]:
print(df.shape)

(25000, 264)


We have a high dimensional data set again. How can we create an effective predictive model quickily on such a complicated data-set?



### Data Proprecessing

Even a quick and dirty classification model requires the transformation of non-numeric feature, in order to enable Python to identify the variable. My method was so 'dirty' that I even included misleading variables in my model. Now I will modify my Kernel to a more completed, accurate one.

In [26]:
opioids=pd.read_csv('./inputs/list_opioids.csv')

In [27]:
name=opioids['Drug Name']
import re
new_name=name.apply(lambda x:re.sub("\ |-",".",str(x)))
columns=df.columns
Abandoned_variables = set(columns).intersection(set(new_name))
Kept_variable=[]
for each in columns:
    if each in Abandoned_variables:
        pass
    else:
        Kept_variable.append(each)

In [28]:
df=df[Kept_variable]
print(df.shape)

(25000, 253)


In [29]:
df.head()[:2]

,NPI,Gender,State,Credentials,Specialty,NPPES Provider Last Name,NPPES Provider First Name,NPPES Provider ZIP Code,NPPES Provider State,Totalclaimcount,...,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,avg_op_supply,op_longer,Opioid_Prescriber,op_prescriber
0,1003002320,M,MS,D.M.D. P.A.,Dentist,EKLUND,D.,39208.0,MS,129,...,0,0,0,0,0,0,4.5,1.0,1,1.0
1,1003004771,F,CO,M.D.,Pulmonary Disease,RIGGLE,ERIN,80120.0,CO,537,...,0,0,16,0,0,0,4.5,0.0,0,0.0


In [30]:
# df['NPPES Provider ZIP Code'] = df['NPPES Provider ZIP Code'].str.replace(',', '')
# df['NPPES Provider ZIP Code'] = df['NPPES Provider ZIP Code'].str.replace(' ', '')
df['NPPES Provider ZIP Code'] = df['NPPES Provider ZIP Code'].astype(float)

In [31]:
df.isnull().sum()

NPI                                  0
Gender                               0
State                                0
Credentials                         82
Specialty                            0
NPPES Provider Last Name          2610
NPPES Provider First Name         2610
NPPES Provider ZIP Code           2610
NPPES Provider State                 0
Totalclaimcount                      0
OpioidClaimCount                     0
ACYCLOVIR                            0
ADVAIR.DISKUS                        0
AGGRENOX                             0
ALENDRONATE.SODIUM                   0
ALLOPURINOL                          0
ALPRAZOLAM                           0
AMIODARONE.HCL                       0
AMITRIPTYLINE.HCL                    0
AMLODIPINE.BESYLATE                  0
AMLODIPINE.BESYLATE.BENAZEPRIL       0
AMOXICILLIN                          0
AMOX.TR.POTASSIUM.CLAVULANATE        0
AMPHETAMINE.SALT.COMBO               0
ATENOLOL                             0
ATORVASTATIN.CALCIUM     

In [32]:
df.fillna(0,inplace = True)

In [33]:
df.columns

Index(['NPI', 'Gender', 'State', 'Credentials', 'Specialty',
       'NPPES Provider Last Name', 'NPPES Provider First Name',
       'NPPES Provider ZIP Code', 'NPPES Provider State', 'Totalclaimcount',
       ...
       'VYTORIN', 'WARFARIN.SODIUM', 'XARELTO', 'ZETIA', 'ZIPRASIDONE.HCL',
       'ZOLPIDEM.TARTRATE', 'avg_op_supply', 'op_longer', 'Opioid_Prescriber',
       'op_prescriber'],
      dtype='object', length=253)

In [34]:
df.drop(['NPPES Provider Last Name', 'NPPES Provider First Name','NPPES Provider ZIP Code',
         'NPPES Provider State'], axis =1, inplace = True)

In [35]:
df.head()[:2]

,NPI,Gender,State,Credentials,Specialty,Totalclaimcount,OpioidClaimCount,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,...,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,avg_op_supply,op_longer,Opioid_Prescriber,op_prescriber
0,1003002320,M,MS,D.M.D. P.A.,Dentist,129,29,0,0,0,...,0,0,0,0,0,0,4.5,1.0,1,1.0
1,1003004771,F,CO,M.D.,Pulmonary Disease,537,0,0,61,0,...,0,0,16,0,0,0,4.5,0.0,0,0.0


In [36]:
train,test = train_test_split(df,test_size=0.3,random_state=42)
print(train.shape)
print(test.shape)

(17500, 249)
(7500, 249)


In [37]:
Categorical_columns=['Gender','State','Credentials','Specialty']
for col in Categorical_columns:
    train[col]=pd.factorize(train[col], sort=True)[0]
    test[col] =pd.factorize(test[col],sort=True)[0]

/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/kirosgebremariam/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [38]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR

y = train['Opioid_Prescriber']

X = train.iloc[:,1:245]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [39]:
train.iloc[:,1:245][:2]

,Gender,State,Credentials,Specialty,Totalclaimcount,OpioidClaimCount,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,ALENDRONATE.SODIUM,...,VENTOLIN.HFA,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE
4913,0,18,517,48,153,36,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9338,0,6,284,25,229,0,0,0,0,0,...,0,0,0,0,0,0,0,14,0,0


In [40]:
X =train.iloc[:,1:245]
y = train['Opioid_Prescriber']

In [41]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

print("Linear Regrssion: ")
print("Five cross-val score means: {}".format(cross_val_score(lr_model, X_train, y_train, cv = 5).mean()))
print("Test score:", lr_model.score(X_test, y_test))
print("-----")

knn_reg = KNeighborsRegressor()
knn_reg_params = {
    'n_neighbors':[1,2,3,4,5,6,7,8,9,10]
}
gs_knn_reg = GridSearchCV(knn_reg, knn_reg_params)
gs_knn_reg.fit(X_train, y_train)

print("k-nearest:")
print("Five cross-val score means: {}".format(cross_val_score(gs_knn_reg, X_train, y_train, cv = 5).mean()))
print("Test score:", gs_knn_reg.score(X_test, y_test))
print("-----")

tree_reg = DecisionTreeRegressor()
tree_reg_params = {
#     'n_neighbors':[1,2,3,4,5,6,7,8,9,10]
}
gs_tree_reg = GridSearchCV(tree_reg, tree_reg_params)
gs_tree_reg.fit(X_train, y_train)

print("Decision Tree Regressor: ")
print("Five cross-val score means: {}".format(cross_val_score(gs_tree_reg, X_train, y_train, cv = 5).mean()))
print("Test score:", gs_tree_reg.score(X_test, y_test))
print("-----")

bag_reg = BaggingRegressor()
bag_reg_params = {}

gs_bag_reg = GridSearchCV(bag_reg, bag_reg_params)
gs_bag_reg.fit(X_train, y_train)

print("Bagging Regressor: ")
print("Five cross-val score means: {}".format(cross_val_score(gs_bag_reg, X_train, y_train, cv = 5).mean()))
print("Test score:", gs_bag_reg.score(X_test, y_test))
print("-----")

rf_reg = RandomForestRegressor()
rf_reg_params = {}

gs_rf_reg = GridSearchCV(rf_reg, rf_reg_params)
gs_rf_reg.fit(X_train, y_train)

print("Random Forest Regressor: ")
print("Five cross-val score means: {}".format(cross_val_score(gs_rf_reg, X_train, y_train, cv = 5).mean()))
print("Test score:", gs_rf_reg.score(X_test, y_test))
print("-----")

ada_reg = AdaBoostRegressor()
ada_reg_params = {}

gs_ada_reg = GridSearchCV(ada_reg, ada_reg_params)
gs_ada_reg.fit(X_train, y_train)

print("AdaBoost Regressor:")
print("Five cross-val score means: {}".format(cross_val_score(gs_ada_reg, X_train, y_train, cv = 5).mean()))
print("Test score:", gs_ada_reg.score(X_test, y_test))
print("-----")

svr = SVR()
svr_params = {
    'C':[.1,1,.1]
}

gs_svr = GridSearchCV(svr, svr_params)
gs_svr.fit(X_train, y_train)

print("Support Vector Regressor:")
print("five cross-val score means: {}".format(cross_val_score(gs_svr, X_train, y_train, cv = 5).mean()))
print("Test score:", gs_svr.score(X_test, y_test))
print("-----")

Linear Regrssion: 
Five cross-val score means: 0.558833316275933
Test score: 0.5913652537140699
-----
k-nearest:
Five cross-val score means: 0.5671962663791562
Test score: 0.5269664604001213
-----
Decision Tree Regressor: 
Five cross-val score means: 0.02158379846227975
Test score: 0.3185779003291218
-----
Bagging Regressor: 
Five cross-val score means: 0.5475622646936321
Test score: 0.6222648709399
-----
Random Forest Regressor: 
Five cross-val score means: 0.5617690592939093
Test score: 0.5484221752906351
-----
AdaBoost Regressor:
Five cross-val score means: -2.2015968271310635
Test score: -1.308676658081274
-----
Support Vector Regressor:
five cross-val score means: -0.006076829101235237
Test score: -0.0060379236471603814
-----


In [42]:
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVR

from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.datasets import samples_generator
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [43]:
from sklearn.metrics import mean_squared_error
import numpy as np
print("RMSE scores")
print("\n")
print("LR:\n------")
print("Train score:", np.sqrt(mean_squared_error(lr_model.predict(X_train), y_train)))
print("Test score:", np.sqrt(mean_squared_error(lr_model.predict(X_test), y_test)))

print("\n")
print("KNN:\n------")
print("Train score:", np.sqrt(mean_squared_error(gs_knn_reg.predict(X_train), y_train)))
print("Test score:", np.sqrt(mean_squared_error(gs_knn_reg.predict(X_test), y_test)))

print("\n")
print("Tree:\n------")
print("Train score:", np.sqrt(mean_squared_error(gs_tree_reg.predict(X_train), y_train)))
print("Test score:", np.sqrt(mean_squared_error(gs_tree_reg.predict(X_test), y_test)))

print("\n")
print("Bag:\n------")
print("Train score:", np.sqrt(mean_squared_error(gs_bag_reg.predict(X_train), y_train)))
print("Test score:", np.sqrt(mean_squared_error(gs_bag_reg.predict(X_test), y_test)))

print("\n")
print("Random Forest:\n------")
print("Train score:", np.sqrt(mean_squared_error(gs_rf_reg.predict(X_train), y_train)))
print("Test score:", np.sqrt(mean_squared_error(gs_rf_reg.predict(X_test), y_test)))

print("\n")
print("AdaBoost:\n------")
print("Train score:", np.sqrt(mean_squared_error(gs_ada_reg.predict(X_train), y_train)))
print("Test score:", np.sqrt(mean_squared_error(gs_ada_reg.predict(X_test), y_test)))

print("\n")
print("SVR:\n------")
print("Train score:", np.sqrt(mean_squared_error(gs_svr.predict(X_train), y_train)))
print("Test score:", np.sqrt(mean_squared_error(gs_svr.predict(X_test), y_test)))

RMSE scores


LR:
------
Train score: 5.196921296239649
Test score: 7.115028225059413


KNN:
------
Train score: 6.440482734791147
Test score: 7.65517192118225


Tree:
------
Train score: 0.0
Test score: 9.187917531814735


Bag:
------
Train score: 3.055023279468626
Test score: 6.840733461092772


Random Forest:
------
Train score: 3.107677497516663
Test score: 7.479546815912617


AdaBoost:
------
Train score: 15.967394170681626
Test score: 16.911827385705465


SVR:
------
Train score: 11.014892519291795
Test score: 11.163905647360451


In [46]:
df.columns

Index(['NPI', 'Gender', 'State', 'Credentials', 'Specialty', 'Totalclaimcount',
       'OpioidClaimCount', 'ACYCLOVIR', 'ADVAIR.DISKUS', 'AGGRENOX',
       ...
       'VYTORIN', 'WARFARIN.SODIUM', 'XARELTO', 'ZETIA', 'ZIPRASIDONE.HCL',
       'ZOLPIDEM.TARTRATE', 'avg_op_supply', 'op_longer', 'Opioid_Prescriber',
       'op_prescriber'],
      dtype='object', length=249)

In [ ]:
#Evaluating  the model

In [47]:
ineligible_noOpioidsPrescriber = (df['Opioid_Prescriber'] == 0) & (df['op_longer'] == 0)
ineligible_yesOpioid_Prescriber = (df['Opioid_Prescriber'] == 0) & (df['op_longer'] == 1)
eligible_noOpioid_Prescriber = (df['Opioid_Prescriber'] == 1) & (df['op_longer'] == 0)
eligible_yesOpioid_Prescriber = (df['Opioid_Prescriber'] == 1) & (df['op_longer'] == 1)
print(
"Ineligible and not Opioid_Prescriber:\t\t", df[ineligible_noOpioidsPrescriber].shape[0],
"\nIneligible and has a Opioid_Prescriber:\t", df[ineligible_yesOpioid_Prescriber].shape[0],
"\nEligible and noOpioid_Prescriber:\t\t", df[eligible_noOpioid_Prescriber].shape[0],
"\nEligible and has prescribed opiods:\t", df[eligible_yesOpioid_Prescriber].shape[0])

Ineligible and not Opioid_Prescriber:		 11123 
Ineligible and has a Opioid_Prescriber:	 0 
Eligible and noOpioid_Prescriber:		 4 
Eligible and has prescribed opiods:	 13331


In [51]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split

In [52]:
from sklearn.preprocessing import StandardScaler

In [54]:
df.head()[:2]

,NPI,Gender,State,Credentials,Specialty,Totalclaimcount,OpioidClaimCount,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,...,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,avg_op_supply,op_longer,Opioid_Prescriber,op_prescriber
0,1003002320,M,MS,D.M.D. P.A.,Dentist,129,29,0,0,0,...,0,0,0,0,0,0,4.5,1.0,1,1.0
1,1003004771,F,CO,M.D.,Pulmonary Disease,537,0,0,61,0,...,0,0,16,0,0,0,4.5,0.0,0,0.0
